<a href="https://colab.research.google.com/github/vmotta/Algebra-Linear-Python/blob/master/geracao_pb_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==1.8.0
#Importar o Tensorflow
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import argparse
from google.protobuf import text_format
from tensorflow.python.platform import gfile
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph

#reset runtimes
def restart_runtime():
  os.kill(os.getpid(), 9)

print('Versão do TensorFlow: ',tf.__version__)
#criação dos diretórios para salvar os checkpoints e o .pb
save_dir = './checkpoint/'
save_pb = './pb/'
save_log = './log/'     

#inicia o tunelamento e mostrará um link
#tbc=TensorBoardColab()

#criação da base de dados
x=np.array([[0.,0.],[0.,1.],[1.,0.],[1.,1.]])#atributos previsores
y=np.array([[0.],[1.],[1.],[0.]])#atributis da classe

#definição das camadas
neuronios_entrada = 2
neuronios_oculta = 3
neuronios_saida = 1

#criação dos pesos das camadas de forma aleatória
w = {'oculta' : tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta]), name='w_oculta',dtype=tf.float32),
    'saida': tf.Variable(tf.random_normal([neuronios_oculta,neuronios_saida]), name='w_saida',dtype=tf.float32)}

#criação dos bias das camadas de forma aleatória
b = {'oculta' : tf.Variable(tf.random_normal([neuronios_oculta]), name='b_oculta',dtype=tf.float32),
    'saida': tf.Variable(tf.random_normal([neuronios_saida]), name='b_saida',dtype=tf.float32)}

#criação dos placesholders
xph = tf.placeholder(tf.float32,[4, neuronios_entrada], name='xph')
yph = tf.placeholder(tf.float32,[4, neuronios_saida], name='yph')

#formulas
camada_oculta = tf.add(tf.matmul(xph,w['oculta']),b['oculta'])
camada_oculta_ativacao = tf.sigmoid(camada_oculta)
camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']),b['saida'])
camada_saida_ativacao = tf.sigmoid(camada_saida)
erro = tf.losses.mean_squared_error(yph, camada_saida_ativacao)
otimizador = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(erro)


#cria os diretórios caso não existam
#if not os.path.exists(save_dir):
#    os.makedirs(save_dir)  
    
if not os.path.exists(save_pb):
    os.makedirs(save_pb)
    
#cria inicialização para iniciar variáveis do tensorflow dentro da sessão 
init = tf.global_variables_initializer()

#inicia sessão tensorflow
with tf.Session() as sess:
    #inicia as variáveis do tensorflow
    sess.run(init)
    #inicia o contador de épocas
    epocas=0
    #inicia o erro médio com valor alto para entrar no while
    erro_medio = 1
    #roda a rede até o erro for pequeno 
    while erro_medio >= 1e-2:
        #inicia o erro médio com 
        erro_medio = 0
        #inicia o feed dos placeholders e roda o otmizador e retorna o erro
        _, custo =sess.run([otimizador, erro], feed_dict = {xph: x, yph: y})
        #a cada 200 epocas ou o erro médio ser zero calcula o erro médio
        if (epocas % 200 == 0) or (erro_medio == 0):
            #calcula o erro médio para cada saída
            erro_medio += custo/4
            #incrementa as épocas
            epocas+=1    
            #print('Epoca: ', epocas, ' -> Erro Médio: ', erro_medio)
    #salva os pesos finais e o bias final        
    w_final, b_final = sess.run([w,b]) 
    
        
# Para limpar as definições de variáveis e operações do grafo do tensorflow
tf.reset_default_graph()   

#criação da base de dados
x=np.array([[0.,0.]])#atributos previsores
y=np.array([[0.]])#atributos previsores
    
#criação dos placesholders
xteste = tf.placeholder(tf.float32,[1, neuronios_entrada], name='xteste')
#output = tf.placeholder(tf.float32,[1, neuronios_saida], name='output')  

#formulas teste de rede
camada_oculta_teste = tf.add(tf.matmul(xteste,w_final['oculta']),b_final['oculta'])
camada_oculta_ativacao_teste = tf.sigmoid(camada_oculta_teste)
camada_saida_teste = tf.add(tf.matmul(camada_oculta_ativacao_teste, w_final['saida']),b_final['saida'])
camada_saida_ativacao_teste = tf.sigmoid(camada_saida_teste, name="output")
#print(camada_saida_ativacao_teste)


saver = tf.train.Saver()

feed_dict_test = {xteste: x}
with tf.Session() as sess:
    sess.run(init)
    #execução da rede de teste
    #print(sess.run(camada_saida_ativacao_teste, feed_dict=feed_dict_test))
    #output = sess.run(camada_saida_ativacao_teste, feed_dict=feed_dict_test)
    sess.run(camada_saida_ativacao_teste, feed_dict=feed_dict_test)
    #print(tf.get_default_graph().get_operations())
    #print(list(a.op.outputs))   
    print(camada_saida_ativacao_teste.op.outputs)
    
    #nome do arquivo de salvamento de log e faz a saida em formato .pbtxt
    filename = 'my_test_model'
       
    # Exporta o checkpoint para SavedModel
    #salva o grafo e eventos da sessão em arquivos de log 
    train_writer = tf.summary.FileWriter('./checkpoint/', sess.graph)
    
    # save the variable in the disk
    saved_path = saver.save(sess, './checkpoint/'+filename+'.ckpt')
    print('model saved in {}'.format(saved_path))

Versão do TensorFlow:  1.8.0


ValueError: ignored

Geração dos nomes dos nós do grafo  para o congelamento

In [0]:
output_node_names = []

for op in tf.get_default_graph().get_operations():
    #print(str(op.name))
    output_node_names.append(str(op.name))
print(output_node_names)    

Função de Congelamento do Grafo completa

In [0]:
# The original freeze_graph function
# from tensorflow.python.tools.freeze_graph import freeze_graph 
# For run in labview IMAQ DL Model Create VI keep constants equals zero
#dir = os.path.dirname(os.path.realpath(__file__))

def freeze_graph(model_dir, output_node_names, constants = 0):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 

    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        # We restore the weights
        saver.restore(sess, input_checkpoint)
        
        if constants == 1:
            print("constantes")
            # We use a built-in TF helper to export variables to constants
            output_graph_def = tf.graph_util.convert_variables_to_constants(
                sess, # The session is used to retrieve the weights
                tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
                output_node_names # The output node names are used to select the usefull nodes
            )
        else:
            print('variaveis')
            output_graph_def = tf.get_default_graph().as_graph_def()

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

    return output_graph_def



In [0]:
freeze_graph('./checkpoint/', output_node_names)

Mostra .PB no TensorFlow

In [0]:
#Importar o Tensorflow
import tensorflow as tf
!pip install --ignore-installed tf-nightly
from google.colab import files

def run_tensorboard(model_filename=''):

    #model_filename ='/content/frozen_model.pb'
    if  model_filename == '':
        uploaded = files.upload()   

    from tensorflow.python.platform import gfile
    with tf.Session() as sess:
       
        with gfile.FastGFile(model_filename, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            g_in = tf.import_graph_def(graph_def)
    LOGDIR='/content/logs'
    train_writer = tf.summary.FileWriter(LOGDIR)
    train_writer.add_graph(sess.graph)

    %load_ext tensorboard
    %tensorboard --logdir ./logs
run_tensorboard('/content/checkpoint/frozen_model.pb')   
#run_tensorboard('/content/checkpoint/frozen_inference_SSDMobilenetV1_graph.pb') 